# LCEL Interface

#### 필수 라이브러리 다운로드

In [79]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai

#### OpenAI API Key 등록

In [80]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-***********************************************"

In [81]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#모델 생성
model = ChatOpenAI()

#프롬프트 생성
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

#LCEL 체인 생성
chain = prompt | model

## Input Schema

In [82]:
# The input schema of the chain is the input schema of its first part, the prompt.
#체인 Schema 확인
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [83]:
#프롬프트 Schema 확인
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [84]:
#모델 스키마 확인
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'strin

## Output Schema

In [85]:
# The output schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage
# Output Schema 확인
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}

## Parallelism

In [86]:
from langchain_core.runnables import RunnableParallel

#체인 생성
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)

#각 변수에 체인 할당
combined = RunnableParallel(joke=chain1, poem=chain2)

In [87]:
%%time
#Chain 1 결과 확인
chain1.invoke({"topic": "bears"})

CPU times: user 66.3 ms, sys: 737 µs, total: 67.1 ms
Wall time: 795 ms


AIMessage(content='Why did the bear bring a flashlight to the party?\nBecause he heard the party was going to be a "beary" good time!', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [88]:
%%time
#Chain 2 결과 확인
chain2.invoke({"topic": "bears"})

CPU times: user 47.4 ms, sys: 0 ns, total: 47.4 ms
Wall time: 601 ms


AIMessage(content='In the forest they roam free,\nMajestic creatures, wild and grand to see.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [89]:
%%time
#결합 체인 결과 확인 
combined.invoke({"topic": "bears"})

CPU times: user 118 ms, sys: 4.9 ms, total: 123 ms
Wall time: 1.08 s


{'joke': AIMessage(content="Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship anymore!", response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'poem': AIMessage(content='In the forest they roam, majestic and wild\nBears of strength, protectors of the child', response_metadata={'finish_reason': 'stop', 'logprobs': None})}

### Parallelism on batches

In [90]:
%%time
#Chain 1 배치 결과 확인
chain1.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: user 93.6 ms, sys: 3.96 ms, total: 97.5 ms
Wall time: 804 ms


[AIMessage(content='Why do bears have fur coats?\n\nBecause they look grizzly without them!', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='Why was the cat sitting on the computer? \nBecause it wanted to keep an eye on the mouse!', response_metadata={'finish_reason': 'stop', 'logprobs': None})]

In [91]:
%%time
#Chain 2 배치 결과 확인
chain2.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: user 86.9 ms, sys: 3.62 ms, total: 90.5 ms
Wall time: 682 ms


[AIMessage(content='In the forest, bears roam free\nMajestic creatures, wild and fierce', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='Whiskers soft, eyes so bright,\nFeline grace in the moonlight.', response_metadata={'finish_reason': 'stop', 'logprobs': None})]

In [92]:
%%time
# 결합 체인 배치 결과 확인 
combined.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: user 210 ms, sys: 12.4 ms, total: 222 ms
Wall time: 1.17 s


[{'joke': AIMessage(content='Why did the bear bring a flashlight to the party?\nBecause he heard it was going to be a "beary" bright night!', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
  'poem': AIMessage(content='Majestic creatures of the wild,\nIn forests deep, they roam and smile.', response_metadata={'finish_reason': 'stop', 'logprobs': None})},
 {'joke': AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
  'poem': AIMessage(content='Whiskers soft, eyes bright,\nPurring through the quiet night.', response_metadata={'finish_reason': 'stop', 'logprobs': None})}]